<a href="https://colab.research.google.com/github/lokeshvirani/Internship-Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1  **Import Libraries**




In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


2  **Load the MNIST Dataset**


*   Loads the MNIST dataset, which contains 60,000 training images and 10,000 test images of handwritten digits 0 to 9.




In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


3 **Expand Dimensions**

In [7]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)


4 **Resize Image**

In [8]:
x_train_resized = np.array([tf.image.resize(image, (32, 32)).numpy() for image in x_train])
x_test_resized = np.array([tf.image.resize(image, (32, 32)).numpy() for image in x_test])


5 **Convert to RGB**

In [9]:
x_train_rgb = np.concatenate([x_train_resized, x_train_resized, x_train_resized], axis=-1)
x_test_rgb = np.concatenate([x_test_resized, x_test_resized, x_test_resized], axis=-1)


6 **Normalize the Images**

In [10]:
x_train_rgb = x_train_rgb.astype('float32') / 255
x_test_rgb = x_test_rgb.astype('float32') / 255


7 **Load ResNet50**

In [11]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


8 **Freeze the Base Model**

In [22]:
base_model.trainable = False


9 **Create Feature Extractor Model**

In [13]:
feature_extractor = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D()
])

10 **Extract Features**

In [14]:
x_train_features = feature_extractor.predict(x_train_rgb)
x_test_features = feature_extractor.predict(x_test_rgb)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 262s 138ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 44s 139ms/step


11 **Standardize Features**

In [15]:
scaler = StandardScaler()
x_train_features = scaler.fit_transform(x_train_features)
x_test_features = scaler.transform(x_test_features)


12 **Prepare One-Class SVM Data**

In [16]:
digit_class = 0
x_train_one_class = x_train_features[y_train == digit_class]


13 **Train One-Class SVM**

In [17]:
oc_svm = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma='auto')
oc_svm.fit(x_train_one_class)


OneClassSVM(gamma='auto', nu=0.1)

14 **Make Predictions**

In [18]:
test_predictions = oc_svm.predict(x_test_features)


15 **Convert Predictions**

In [41]:
test_predictions_binary = (test_predictions == 1).astype(int)


In [42]:
# Create binary ground truth for the test set (1 if the class is the digit we
# trained on, otherwise 0)
y_test_binary = (y_test == digit_class).astype(int)

In [43]:
print(classification_report(y_test_binary, test_predictions_binary))

              precision    recall  f1-score   support

           0       0.98      0.41      0.58      9020
           1       0.14      0.91      0.25       980

    accuracy                           0.46     10000
   macro avg       0.56      0.66      0.41     10000
weighted avg       0.89      0.46      0.55     10000

